In [1]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import os

In [2]:
# Load trained model and tokenizer
model_path = "./models/t5_task_b_model_v2"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

In [3]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [4]:
# Load test data
test_file = "../dataset/task_b+c/data/challenge_data/clinicalnlp_taskB_test1.csv"
test_df = pd.read_csv(test_file)

In [5]:
# Preprocess: add input formatting
test_df["input_text"] = "Dialogue: " + test_df["dialogue"].str.replace(r"\s+", " ", regex=True).str.strip()

In [6]:
# Generate predictions
model.eval()
predictions = []

In [7]:
for text in tqdm(test_df["input_text"], desc="Generating notes..."):
    input_ids = tokenizer(text, return_tensors="pt", truncation=True, max_length=768).input_ids.to(device)
    output = model.generate(
        input_ids,
        max_length=512,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    predictions.append(decoded.strip())

Generating notes...: 100%|██████████| 40/40 [01:23<00:00,  2.08s/it]


In [8]:
# Create submission DataFrame
submission_df = pd.DataFrame({
    "EncounterID": test_df["encounter_id"],
    "SystemOutput": predictions
})

In [9]:
# Save predictions
os.makedirs("../outputs", exist_ok=True)
output_path = "../outputs/taskB_run1.csv"
submission_df.to_csv(output_path, index=False)
print("Saved predictions to:", output_path)

Saved predictions to: ../outputs/taskB_run1.csv


In [11]:
import pandas as pd

# Load both files
df_gold = pd.read_csv("../dataset/task_b+c/data/challenge_data/clinicalnlp_taskB_test1.csv")
df_sys = pd.read_csv("../outputs/taskB_run1.csv")

print("GOLD COLUMNS:", df_gold.columns.tolist())
print("SYS COLUMNS:", df_sys.columns.tolist())

GOLD COLUMNS: ['dataset', 'encounter_id', 'dialogue', 'note']
SYS COLUMNS: ['EncounterID', 'SystemOutput']
